# Semantic Kitti vizualisation

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'
pv.set_plot_theme("document")

from torch_points3d.datasets.segmentation.semantickitti import SemanticKittiDataset, SemanticKitti
from torch_points3d.datasets.segmentation import IGNORE_LABEL

## Load Scannet dataset

In [ ]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.load(os.path.join(DIR,'conf/data/segmentation/semanticKitti.yaml'))

In [ ]:
dataset_options.data.dataroot = os.path.join(DIR,"data")
dataset = SemanticKittiDataset(dataset_options.data)
print(dataset)

## Visualise the data

In [ ]:
def buil_cmap():
    mapping = np.linspace(IGNORE_LABEL, 20, 256)
    newcolors = np.zeros((256, 3))
    for i, raw_label in SemanticKitti.LEARNING_MAP_INV.items():
        color = SemanticKitti.COLOR_MAP[raw_label]
        newcolors[mapping >= i] = np.asarray(color) / 255.
    return ListedColormap(newcolors)
cmap = buil_cmap()

In [ ]:
def load_random_data(event):
    i = np.random.randint(0, len(dataset.train_dataset))
    sample = dataset.train_dataset[i]
    xyz = sample.pos.numpy()
    pl = pv.Plotter(notebook=True)
    point_cloud = pv.PolyData(xyz)
    point_cloud['label'] = sample.y.numpy()
    pl.add_points(point_cloud,cmap=cmap) 
    
    pan.object = pl.ren_win
    return point_cloud

In [ ]:
pl = pv.Plotter(notebook=True)
pan = pn.panel(pl.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=True,)

In [ ]:
button = pn.widgets.Button(name='New Random', button_type='primary')
button.on_click(load_random_data)

In [ ]:
dashboard = pn.Row(
    pn.Column('## Visualise sample',button),
    pan
)

In [ ]:
dashboard